# Graph Traversals (Cont'd)

 The Implementation of DFS went by very quickly earlier today, so we'll take a second look with a few more examples.

In [1]:
%config InteractiveShell.ast_node_interactivity="none"

In [2]:
%run boaz_utils.ipynb

## Trees
A **tree** is an undirected graph that contains no cycles.  This means that every pair of nodes has a unique path between them.  This property makes it very suitable for trying to understand the DFS algorithm, because we do not need the visited dictionary.

So we'll take a look at some computations on trees to see the power of recursion, as well as gain a little more insight into the DFS traversal.


### A Binary Tree as a Directed Graph
Let us consider directed graphs that otherwise resemble trees.  
- Each node will have at most 2 out-neighbours.  
- Each node will also have exactly 1 in-neighbour, except for one node called the **root**, which will have no in-neighbour. 
- A node with zero out-neighbours is called a **leaf**.

Here are some examples of graphs that would be regarded as binary trees under this definition. (Developed collaboratively on the board during lecture)

For expediency, let us use a list instead of a dictionary to represent the adjacency list of a graph.  Here, we'll assume that the nodes are indexed from 0 (and don't have names that the user knows about).

(We are doing this just so that we can type in examples faster).

For example:
`[[1], [2, 3], [], []]` 
represents the graph $\langle V, E\rangle$, where:
- $V = \{0, 1, 2, 3\}$
- $E = \{(0, 1), (1, 2), (1, 3) \}$

In [3]:
def graph_nodes(graph):
    """Return a list of all the nodes in graph"""
    return range(len(graph))

def graph_nbrs(graph, node):
    """Return a list of the neighbours of node"""
    return graph[node]

def graph_size(graph):
    return len(graph)

#### Count Descendants
Let us develop a program to count the number of nodes reachable in the tree, from a given start node.

In a tree, such reachable nodes are also called _descendants_ of the start node.

In [4]:
def count_reachable(tree, node):
    """Return the number of nodes reachable from node in tree"""
    nbrs = graph_nbrs(tree, node)
    if len(nbrs) == 0:
        return 1
    elif len(nbrs) == 1:
        return count_reachable(tree, nbrs[0]) + 1
    else:
        return count_reachable(tree, nbrs[0]) + \
               count_reachable(tree, nbrs[1]) + 1


In [5]:
g1 = [[1], [2, 3], [], []]
print(count_reachable(g1, 0))  # Should print 4

4


In [ ]:
g2 = [[1, 2], [3, 4], [5], [6, 7], [], [8, 9], [], [10], [], [], [11], []]
print(count_reachable(g2, 0))  # Should print 12
print(count_reachable(g2, 2))  # Should print 4

In [6]:
# Develop an example on the board together
g0 = [[1, 2], [3, 4], [5], [], [], [6], []]
print(count_reachable(g0, 0))
print(count_reachable(g0, 1))

7
3


#### Count Leaves
Try to develop a program that would count the number of _leaves_ that were reachable from the root.

In [8]:
def count_leaves(tree, node):
    """Return the number of leaves reachable from node in tree"""
    nbrs = graph_nbrs(tree, node)
    if len(nbrs) == 0:
        return 1
    elif len(nbrs) == 1:
        return count_leaves(tree, nbrs[0])
    else:
        return count_leaves(tree, nbrs[0]) + \
               count_leaves(tree, nbrs[1])


In [9]:
print(count_leaves(g0, 0))


3


### General Trees

If you have a general tree, how would you count the number of reachable nodes?

In [ ]:
def count_reachable_gen(tree, node):
    """Return the number of nodes reachable from node"""
    nbrs = graph_nbrs(tree, node)
    if len(nbrs) == 0:
        return 1
    else:
        count = 1
        for nbr in nbrs:
           count += count_reachable_gen(tree, nbr)
        return count

In [ ]:
print(count_reachable_gen(g1, 0))  # Should be 4

In [ ]:
print(count_reachable_gen(g2, 0))  # Should still be 12

In [ ]:
g3 = [[1, 2, 3], [4], [5, 6], [7], [8], [], [], [], []]
print(count_reachable_gen(g3, 0))  # Should print 9

Observe how there is a recursive call _inside_ a for loop in `count_reachable_gen`.  It seems scary at first if we come at it without preparation, but when we see how we built up to it, we can recognise the for loop as generalizing over the number of out-neighbours, and the recursive call is just the wishful thinking that we are carrying out per out-neighbour.

What else we do inside and outside of the for loop is all about putting together the results of those "wishful thinking" processes (ie recursive calls) to produce the final answer that we were after. 

## DFS Implementation
Now we are ready to re-examine the DFS implementation. 

Here is the main code from earlier:

In [ ]:
def init_visited(graph):
    """Return a dictionary mapping each node in graph to False """
    result = {}
    for node in graph_nodes(graph):
        result[node] = False
    return result

# This is just to get started
def dfs_find_reachable(g, start):
    visited = init_visited(g)
    reachable = []
    dfs_explore_reachable(g, start, visited, reachable)
    return reachable

# This one will do the hard work
def dfs_explore_reachable(g, node, visited, reachable):
    visited[node] = True
    reachable.append(node)
    for nbr in graph_nbrs(g, node):
        if not visited[nbr]:
            dfs_explore_reachable(g, nbr, visited, reachable)

### Discussion

Counting reachable nodes is the same thing as exploring, but we are also keeping track of a count.  In our discussion so far, we have limited ourselves to trees, which did not have cycles, so it was impossible to revisit the same node twice, if we started out on different out-neighbours from the root.

In a general graph, this will not be the case though.  It might be possible that if we start at one node, $u$, and explore from its first out-neighbour, we could get to some node $v$; and then later, when exploring from $u$'s second out-neighbour, we also get to the same node $v$.  If we counted reachable nodes in the way we did for trees, we would end up double counting $v$.  So, to avoid this, we have to keep track of a "register" of which nodes have already been visited.  We need to make sure that it is available to every recursive call, so that as it is updated, other recursive calls see those updates.

Looking back at our DFS implementation, we can see a strong similarity to the process for counting reachable nodes. The main difference is the existence of a record of the nodes that have been visited (the `visited` dictionary), which is passed to every recursive call.  Notice also that we consult that record to avoid making the recursive call at any out-neighbours that have already been accounted for (i.e. visited).

## Binary Tree Abstraction
- A fundamental data structure in computer science.
- Many other data structures built on top of it
- Offers opportunity for $O(\log n)$ time performance instead of $O(n)$ for some problems.

### Definition
- Either:
  - Empty, OR
  - Contains a node (the root) AND 2 sub-trees (each of which is also a Binary Tree)

(A _binary tree_ is a **recursive data structure**.)
  
### Application
- Used to organise information
- Node "contains" useful data (e.g. key of a dictionary)
- Node is anonymous (unlike general graph)

## Cautionary Note
This approach to implementing a binary tree is **unconventional**. Since binary trees are restricted versions of graphs, we can use much simpler building blocks to implement them.

Nevertheless, the exercise here can be a valuable lesson in the power of abstraction.

## An ADT for Binary Trees
Having already developed code to handle general graphs, we could build an abstraction for Binary Trees on top of our implementation of graphs.

We can define functions that would encourage a user of our version of binary trees to not focus on the fact that we are building on top of a more general representation (i.e. for general graphs).

### Operations supported by Binary Trees
- **Constructor**: 
  - `mk_binary_tree(root_value, left_subtree, right_subtree)`
     - (We'll assume we get a populated tree via a graph, so we won't bother with this)
  - `graph2tree(graph, root, values)`: Treat the graph as a binary tree rooted at the given node. `values` is a list associating the node indexes with their data.
- **Selectors**:
  - `root(tree)`: return the root value of tree
  - `left_subtree(tree)`: return the left subtree of tree
  - `right_subtree(tree)`: return the right subtree of tree
- **Predicates**:
  - `is_empty(tree): return True if the tree is empty
- **Constants**:
  - `EMPTY`  (the empty tree)
  

In [ ]:
# Complete the following ADT functions
def graph2tree(graph, node, values):
    """Given the neighbours of a node, return the one on its left"""
    pass

def root(tree):
    """Given the neighbours of a node, return the one on its right"""
    pass

def left_subtree(tree):
    pass

def right_subtree(tree):
    pass

def is_leaf(tree):
    pass



In our context, we are assuming we have an underlying graph, so the tree would never technically be empty, but we still need to implement `is_empty` so that algorithms that depend upon it would still work. Can you think of a way to represent the empty tree, and therefore to implement `is_empty`? 

(Hint: There are many possible good answers)

In [ ]:
# EMPTY = ???

def is_empty(tree):
    pass